In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import zarr
import holoviews as hv
import matplotlib.pyplot as plt
from functools import partial
from cytoolz import *
import hvplot.pandas
import scipy
import re
import os
from tqdm.auto import tqdm

IDX = pd.IndexSlice

# Functions

In [ ]:
def RevImage(img, **kwargs):
    return _RevImage(hv.Image, img, **kwargs)


def _RevImage(cls, img, **kwargs):
    return cls(img[::-1], bounds=(0, 0, img.shape[1], img.shape[0])).options(
        invert_yaxis=True
    )


def _horiz_trench_img(img):
    return RevImage(img.T).options(backend="bokeh", height=120, width=400)


def _trench_img(img):
    return RevImage(img).options(backend="bokeh", height=400, width=120)

In [ ]:
def _load_measurements(base_filename, kind):
    labelwise_filename = base_filename + ".{}.parquet".format(kind)
    if not os.path.exists(labelwise_filename):
        return
    with pa.OSFile(labelwise_filename, "rb") as f:
        res = pq.ParquetFile(f).read(use_pandas_metadata=True, use_threads=False)
    res = res.to_pandas()
    return res
    # labelwise_df = data_io.read_parquet(labelwise_filename, progress_bar=None)
    # mask_labelwise_df = data_io.read_parquet(base_filename+'.mask_labelwise.parquet')
    # trenchwise_df = data_io.read_parquet(base_filename+'.trenchwise.parquet')
    # return labelwise_df#.to_pandas()


def load_measurements(
    parquet_filename, nthreads=False, progress_bar=tqdm, filename=None
):
    positions = [
        re.match(r".*pos(\d+)\.labelwise\.parquet", path)
        for path in os.listdir(parquet_filename)
    ]
    positions = [int(m.group(1)) for m in positions if m is not None]
    print(positions)
    if progress_bar is not None:
        positions = progress_bar(positions, total=len(positions))
    res = {}
    for pos in positions:
        key = (filename, pos)
        base_filename = os.path.join(parquet_filename, "pos{:d}".format(pos))
        # res[key] = _load_measurements(base_filename, kind)
        labelwise_filename = base_filename + ".labelwise.parquet"  # .format(kind)
        if not os.path.exists(labelwise_filename):
            continue
        with pa.OSFile(labelwise_filename, "rb") as f:
            labelwise_df = pq.ParquetFile(f).read(
                use_pandas_metadata=True, use_threads=False
            )
        labelwise_df = labelwise_df.to_pandas()
        res[key] = labelwise_df
    return sorted(res)

In [ ]:
# dat = load_measurements(nd2_filename+'.measurements')

# Traces

In [ ]:
# nd2_filename = '/home/jqs1/scratch/190504/basilisk/190504_YFP_mScarlet_repressilators002.nd2'
nd2_filename = (
    "/home/jqs1/scratch/190504/basilisk/190504_YFP_mScarlet_repressilators_fast.nd2"
)

In [ ]:
dat = pq.read_pandas(
    nd2_filename + ".measurements/pos177.labelwise.parquet"
).to_pandas()

In [ ]:
dat.head()

## Segmentation

In [ ]:
root = zarr.open_group(
    store=zarr.LMDBStore(nd2_filename + ".images/pos177.zarr", readonly=True)
)

In [ ]:
trench_set = 1
trench_idx = 61
raw_stack = np.asarray(root["raw"][trench_set][trench_idx]["CFP"])
label_stack = np.asarray(root["segmentation"][trench_set][trench_idx]["CFP"])

In [ ]:
t = 0
_trench_img(raw_stack[:, :, t]) + _trench_img(label_stack[:, :, t]).redim(z="label")

In [ ]:
(
    hv.HoloMap({t: _trench_img(raw_stack[:, :, t]) for t in range(raw_stack.shape[-1])})
    + hv.HoloMap(
        {
            t: _trench_img(label_stack[:, :, t]).redim(z="label")
            for t in range(raw_stack.shape[-1])
        }
    )
)